## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [2]:
import glob
import sys
import numpy as np
import scipy as sp
import scipy.io
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib import mlab
import matplotlib.gridspec as gridspec
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import pandas as pd


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    ### return the since the third row
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def create_features(indice, ppg, accx, accy, accz, min_freq_v, max_freq_v, fs=125, verbose=False, plot_frequency_ppg = False, plot_frequency_acc = False):
    """
    Apply Troika process to the pppg and accelerometer signals in order to extract feature for our trained dataset.
    
    
    :param indice: index of the iteration realted to the windows time.
    :param ppg: slice PPG signal.
    :type ppg: ndarray
    :param accx: accelerometer signal on the x-axis
    :type accx: ndarray
    :param accy: accelerometer signal on the y-axis
    :type accy: ndarray
    :param accz: accelerometer signal on the z-axis
    :type accz: ndarray
    :param min_freq_v: min frequency allowed in our descompose signal.
    :type min_freq_v: float
    :param max_freq_v: max frequency allowed in our descompose signal.
    :type max_freq_v: float
    :param fs:  sampling rate
    :type  fs: integer
    :param verbose: show the value of ground truth for the slice window.
    :type verbose: bool
    :param plot_frequency_ppg: plot the frequency domain of the ppg signal for the segment of the signal.
    :type  plot_frequency_ppg: bool
    :param plot_frequency_acc: plot the frequency domain of the acc signal for the segment of the signal.
    :type  plot_frequency_acc: bool
    """
    
    #############################################################################
    #### First Band pass Filtering all the signal PPG, and acceleration 
    ##  Use the 40-240BPM range to create your pass band.
    #############################################################################
    ppg = bandpass_filter(ppg, min_freq = min_freq_v, max_freq = max_freq_v)
    accx = bandpass_filter(accx, min_freq = min_freq_v, max_freq = max_freq_v)
    accy = bandpass_filter(accy, min_freq = min_freq_v, max_freq = max_freq_v)
    accz = bandpass_filter(accz, min_freq = min_freq_v, max_freq = max_freq_v)
    
    # Aggregate accelerometer data into single signal
    accy_mean = accy - np.mean(accy) # Center Y values
    acc_mag_unfiltered = np.sqrt(accx**2 + accy_mean**2 + accz**2)
    acc_mag = bandpass_filter(acc_mag_unfiltered, min_freq = min_freq_v, max_freq = max_freq_v)
  
    #############################################################################
    #### Signal Decomposition ##################################################
    #############################################################################
    fft_len = len(ppg)*4
    
    fft_ppg_initial = np.fft.rfft(ppg)
    freqs_original = np.fft.rfftfreq(len(ppg), 1 / fs)
  
    fft_acc_original = np.fft.rfft(acc_mag)
    freqs_acc_original = np.fft.rfftfreq(len(acc_mag), 1 / fs)
    
    freqs_initial = np.fft.rfftfreq(fft_len, 1 / fs)
    low_freqs = (freqs_initial >= (min_freq_v)) & (freqs_initial <= (max_freq_v))
    
    ### get magnitude
    mag_freq_ppg, fft_ppg = fourier_transform(ppg, freqs_initial, low_freqs, fft_len)
    mag_freq_acc, fft_acc = fourier_transform(acc_mag, freqs_initial, low_freqs, fft_len)    
    
    #############################################################################
    ### Find Peaks ##########################################################
    peaks_ppg = find_peaks(mag_freq_ppg, height=30, distance=1)[0]
    peaks_acc = find_peaks(mag_freq_acc, height=30, distance=1)[0]
    #############################################################################
     
    if plot_frequency_ppg:
        if indice < 2:
            gs = gridspec.GridSpec(2, 2)
            plt.figure()
            ax = plt.subplot(gs[0, 0])
            ax.set_title(f'Initial Frequency PPG {indice}')
            ax.plot(freqs_original, fft_ppg_initial)
            ax.set_xlabel('Frequency (Hz)')
        
            ax = plt.subplot(gs[0, 1])
            ax.set_title(f'Low Frequency PPG {indice}')
            ax.plot(freqs_initial[low_freqs], mag_freq_ppg)
            ax.set_xlabel('Frequency (Hz)')   
            
            
            ### Peaks plot            
            ax = plt.subplot(gs[1, :])
            ax.set_title(f'Find Peaks {indice}')
            ax.plot(mag_freq_ppg)
            ax.plot(peaks_ppg, mag_freq_ppg[peaks_ppg], '+r', ms = 10)
            ax.set_xlabel('Time')              
            plt.show()      
    
    
    if plot_frequency_acc:
        if indice < 2:
            gs = gridspec.GridSpec(2, 2)
            plt.figure()
            ax = plt.subplot(gs[0, 0])
            ax.set_title(f'Initial Frequency ACC {indice}')
            ax.plot(freqs_acc_original, fft_acc_original)
            ax.set_xlabel('Frequency (Hz)')
        
            ax = plt.subplot(gs[0, 1])
            ax.set_title(f'Low Frequency ACC {indice}')
            ax.plot(freqs_acc_original[low_freqs], mag_freq_acc)
            ax.set_xlabel('Frequency (Hz)')   
            
            
            ### Peaks plot            
            ax = plt.subplot(gs[1, :])
            ax.set_title(f'Find Peaks ACC {indice}')
            ax.plot(mag_freq_ppg)
            ax.plot(peaks_acc, mag_freq_acc[peaks_acc], 'xg', ms = 10)
            ax.set_xlabel('Time')              
            
            plt.show()       
    
    
    ppg_feature = freqs_original[np.argmax(fft_ppg_initial)]
    acc_feature = freqs_acc_original[np.argmax(fft_acc_original)]
    

    return (np.array([ppg_feature, acc_feature, len(peaks_ppg), len(peaks_acc)])), freqs_initial,  fft_ppg
                

        
def Confidence(estimation, freqs_initial, fft_ppg, min_freq_v):        
    """
    Calculate the confidence base on the summing of the frequency spectrum around the estimation of the heart rate 
    and dividing it by the sum of the entire spectrum.
    
    
    :param estimation: heart rate estimation
    :type estimation: float
    
    :param freqs_initial: the Discrete Fourier Transform sample frequencies
    :type freqs_initial: ndarray
    
    :param fft_ppg: Compute fourier transformation
    :type fft_ppg: ndarray
    
    :param min_freq_v: min frequency allowed in our descompose signal.
    :type min_freq_v: float
    
    :return: confidence value
    :rtype: float
    """
    #############################################################################
    #### Confidence ############################################################
    #############################################################################
    chosen_freq = estimation / 60.0
    
    win_freqs = (freqs_initial >= chosen_freq - min_freq_v) & (freqs_initial <= chosen_freq + min_freq_v)
    abs_fft_ppg = np.abs(fft_ppg)

    # Sum frequency spectrum near pulse rate estimate and divide by sum of entire spectrum
    confidence = np.sum(abs_fft_ppg[win_freqs])/np.sum(abs_fft_ppg)

    return (estimation, confidence)


def fourier_transform(x, freqs, low_freqs, fft_len):
    ''' Compute and return FFT and magnitude of FFT for given low frequencies
 
    :param x: input signal to transform
    :type x: ndarray
    :param freqs: full list of FFT frequency bins.
    :type freqs: integer ndarray
    :param low_freqs: low frequency bins between 40 BPM and 240 BPM
    :type low_freqs: boolean ndarray
    :param fft_len: length of FFT to compute
    :type fft_len: integer  
    :return:
    mag_freq_x: magnitude of lower frequencies of the FFT transformed signal
    fft_x: FFT of normalized input signal
    '''
    # Take an FFT of the mean normalized signal
    norm_x = (x - np.mean(x))/(max(x)-min(x))
    fft_x = np.fft.rfft(norm_x, fft_len)

    # Calculate magnitude of the lower frequencies
    mag_freq_x = np.abs(fft_x)[low_freqs]
    return mag_freq_x, fft_x


def bandpass_filter(signal, fs = 125, min_freq = 40/60.0, max_freq = 240/60.0):
    """
    Filter the signal between the min_freq and max_freq
    
    
    :param signal: signal from PPG or Accelerometer
    :type signal: ndarray
    
    :param fs: sampling rate
    :type fs: integer
    
    :param min_freq:  min frequency allowed in our descompose signal.
    :type min_freq: float
    
    :param max_freq:  max frequency allowed in our descompose signal.
    :type max_freq: float
    
    :return:  band pass signal
    :rtype: ndarray
    """
    
    pass_band = (min_freq, max_freq)
    b, a = scipy.signal.butter(3, pass_band, btype = 'bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)


def TrainModel():
    """
    Train the model base on features extracted from our PPG and accelerometer signal.
    :return: Model trained over 80% sample of the dataset.
    """
    data_fls, ref_fls = LoadTroikaDataset()

    features_list, target_list = [], []
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):    
        # Load data using LoadTroikaDataFile
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        Fs = 125 # Troika data has sampling rate of 125 Hz
        # Compute pulse rate estimates and estimation confidence.

        min_freq_val = 40/60.0
        max_freq_val = 240/60.0    

        #######################################
        ######################################
        # Load data using LoadTroikaDataFile
        window_size = 8*Fs # Ground truth BPM provided in 8 second windows
        window_shift = 2*Fs # Successive ground truth windows overlap by 2 seconds

        reference = sp.io.loadmat(ref_fl)

        errs = []
        confs = []

        # For each 8 second window, compute a predicted BPM and confidence and compare to ground truth
        offset = 0
        
        for indice, eval_window_idx in enumerate(range(len(reference['BPM0']))):
            window_start = offset
            window_end = window_size + offset
            offset += window_shift

            ppg_window = ppg[window_start:window_end]
            accx_window = accx[window_start:window_end]
            accy_window = accy[window_start:window_end]
            accz_window = accz[window_start:window_end]
            groundTruthBPM = reference['BPM0'][eval_window_idx][0]

            feature, _ , _ = create_features(indice, ppg_window, accx_window, accy_window, accz_window, 
                                        min_freq_val, max_freq_val, Fs, verbose = False, 
                                        plot_frequency_ppg = False, plot_frequency_acc = False)            

            features_list.append(feature)
            target_list.append(groundTruthBPM)

    features = np.array(features_list)
    target =  np.array(target_list)
    
    dffeatures = pd.DataFrame(features)
    
    model_rf = RandomForestRegressor(n_estimators = 350, max_depth = 15)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 2021)
    
    model_rf.fit(X_train, y_train)
    return model_rf    


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    :return: Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    :rtype: 
    """
    ## Trained Model
    global model_ml
    model_ml = TrainModel()
    
    
    ### Test Algorithm
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
        
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Get the heart rate estimation and the confidence values for an specific data file.
    
    :param data_fl: path data of the subjects
    :param ref_fl: path data of the grounth truth
    
    
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    Fs = 125 # Troika data has sampling rate of 125 Hz
    # Compute pulse rate estimates and estimation confidence.
    
    min_freq_val = 40/60.0
    max_freq_val = 240/60.0
    
    #######################################
    ######################################
    # Load data using LoadTroikaDataFile
    window_size = 8*Fs # Ground truth BPM provided in 8 second windows
    window_shift = 2*Fs # Successive ground truth windows overlap by 2 seconds
    
    reference = sp.io.loadmat(ref_fl)

    errs = []
    confs = []
    
    # For each 8 second window, compute a predicted BPM and confidence and compare to ground truth
    offset = 0
    
    
    for indice, eval_window_idx in enumerate(range(len(reference['BPM0']))):
        # Set verbose to True to visualize plot analysis
        verbose = True
        # verbose = True if eval_window_idx == 28 else False
    
        window_start = offset
        window_end = window_size+offset
        offset += window_shift
        
        if verbose:
            print(f"Win start,end: {window_start}, {window_end}")

        ppg_window = ppg[window_start:window_end]
        accx_window = accx[window_start:window_end]
        accy_window = accy[window_start:window_end]
        accz_window = accz[window_start:window_end]
        
        groundTruthBPM = reference['BPM0'][eval_window_idx][0]
    
        feature, freqs_initial,  fft_ppg  = create_features(indice, ppg_window, accx_window, accy_window, accz_window, 
                                    min_freq_val, max_freq_val, Fs, verbose = verbose, 
                                    plot_frequency_ppg = False, plot_frequency_acc = False)
        
        
        estimation_hr =  model_ml.predict(np.reshape(feature, (1, -1)))[0]
        
        if verbose:
            print(f'Ground Truth BPM: {groundTruthBPM}')
            print(f'Prediction BPM {estimation_hr}')

            
        estimation_hr, conf = Confidence(estimation_hr, freqs_initial, fft_ppg, min_freq_val)    
        predError = groundTruthBPM - estimation_hr
        errs.append(predError)
        confs.append(conf)    
    
    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.array(errs), np.array(confs)  # Dummy placeholders. Remove
    return errors, confidence



In [3]:
import scipy.signal
sth = Evaluate()

Win start,end: 0, 1000
Ground Truth BPM: 74.33920704845815
Prediction BPM 90.03269772358979
Win start,end: 250, 1250
Ground Truth BPM: 76.35746606334841
Prediction BPM 95.7958997802082
Win start,end: 500, 1500
Ground Truth BPM: 77.14285714285714
Prediction BPM 94.26507233370029
Win start,end: 750, 1750
Ground Truth BPM: 74.66814159292035
Prediction BPM 97.48102778071645
Win start,end: 1000, 2000
Ground Truth BPM: 72.58064516129032
Prediction BPM 81.37918328726016
Win start,end: 1250, 2250
Ground Truth BPM: 71.68458781362007
Prediction BPM 88.8749572158421
Win start,end: 1500, 2500
Ground Truth BPM: 72.89416846652267
Prediction BPM 115.0685556419423
Win start,end: 1750, 2750
Ground Truth BPM: 73.449401523395
Prediction BPM 96.70456130418694
Win start,end: 2000, 3000
Ground Truth BPM: 75.33482142857142
Prediction BPM 127.0361410593432
Win start,end: 2250, 3250
Ground Truth BPM: 76.84426229508196
Prediction BPM 144.54905122510942
Win start,end: 2500, 3500
Ground Truth BPM: 79.599056603773

In [4]:
sth

8.8514712875202122

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

### Code Description:

#### Programming Language:
- Python >=3.6

#### Dependencies:
- numpy
- scipy
- matplotlib


####  Run the algorithm:
From the main directory:

- Step 1 - Define the path for the DATA AND REF folders which should be set up inside the function LoadTroikaDataset() in the variable `data_dir`.

- Step 2 - Run the function:
```python
result = Evaluate()
```
Optionally:
- In the function create_features, we can set up two parameters to `True` for plotting the frequencies and peaks. `plot_frequency_ppg`,  `plot_frequency_acc`

```python
feature, freqs_initial,  fft_ppg  = create_features(indice, ppg_window, accx_window, accy_window, accz_window, 
                            min_freq_val, max_freq_val, Fs, verbose = verbose, 
                            plot_frequency_ppg = True, plot_frequency_acc = True)
```

### Data Description:
The data contains signals recorded from  wrist devices when the subjects were doing intensive exercise, the signals are contaminated by intentional/unintentional movements of the arm.

Each file contains the following signals:
- 3-axis accelerometer signal 
- PPG signal
    
    
    
    
    
### Algorithm Description:

#### How the algorithm works:

- Load data: Load to memory the PPG signal and accelerometer data from 12 subjects.


- Create Features:
    - Band Pass Filtering: Attenues certain frequencies in a specific range. In this case we apply a range between 40 - 240 BPM. It removes some motion artifacts.

    - Signal Decomposition: It descompose our signal using Fast Fourier transformation. Then the components related to noise and interference are removed. The final signal can be reconstructed base on the components that were not removed.

    - Find peaks: Detect the peaks in our signal that was bandpassed. 

- Train Model: the dataset is splitted assigning 20% of the data for test and 80% for training.

- Test Model: The model is tested by running the algorithm over the whole dataset.

- Calculate confidence: The the confidence is calculate base on the sum of the frequency spectrum near the pulse rate estimate and dividing it by the sum of the entire spectrum. This output is produce every two seconds.

- Agregate Error metrics: Computes the MAE at 90% availability. 


![Troika Flow](img/flow_motion.png)

#### The specific aspects of the physiology that it takes advantage of
- Wrist Type PPG signal: The ppg signal can be use to measure the Heart rate of a subject, this signal is collected from a non invasive method where the signal measure the volumetric variation of the blood circulation.

- Accelerometer signal:  Measures the accelaration forces in different axis, for our project we need to differentiate between unintentional movements thaht generates noise in ur signal to predict the next value of the heart rate.

#### Description of the algorithm outputs

#### Caveats on algorithm outputs
The dataset was build base on the data collected of 12 subjects performing intensive physical exercise (PPG, Accelerometer signal), whose activity is similar. This similarity could lead the algorithm to perform better than it really is if it tested in other subjects whose activity differ from the data that was used. 

#### Common failure modes
- The PPG signal is vulnerable to motion artifacts such as the device not being well-attach to the wrist of the subject. It might add aditional noise to the signal.
- The algorithm was developed to perform well under normal conditions, the algorithm has not been tested under water, high altitude or other extreme condition.


####  Algorithm Performance:

- The algorithm selected was RandomForest because it get the lowest measures of error in comparison to Ridge Regression and Support Vector Machine which exceeds MAE of 15

Random Forest Parameters:
- max_depth = 15 
- estimators = 350

Train test split
- test_percentage = 0.2
- train_percetange = 0.8


- The performance was given by using the Mean absolute Error.

$
MAE = \frac{1}{n} * \sum_{i {=} 1}^{n} \left | x_{i} - x\right | 
$

- The model performs obtaining a `MAE` of **9.01** over the entire dataset comparing a 2 seconds ground truth.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 